# Google Drive Mount


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWgTHgQ082vXCycHZw9AG6_3krywaPWTUNJkuPPA3NEg35r00eLLhXg
Mounted at /content/drive


# Direcoty & Libraries

In [ ]:
%cd /content/drive/MyDrive/Deep_learning/Deep\ Learning

/content/drive/.shortcut-targets-by-id/1D1ZlxwJeesVJ3NPzK7AzVApyX6KyqwBA/Deep_learning/Deep Learning


In [ ]:
#Importing Libraries and Packages
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, IterableDataset

# calculate train time, writing train data to files etc.
import os 
import logging
import pandas as pd
import numpy as np
import time
from pathlib import Path
from random import randint
from scipy import signal
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import scipy.io as sio

import pdb

# Dataset

In [32]:
class CoolDataset(IterableDataset):

    def __init__(self, dir_path: str, seq_length: str, input_size: str, samples_per_events: str, convolution=True):

        # Opens .csv file and read headers
        # Needs to decide what inputs to have and work on HS or FO

        super().__init__()

        self.files = tuple(Path(dir_path).glob("**/*.csv"))
        self.seq_length = seq_length
        self.input_size = input_size
        self.SAMPLES_PER_EVENT = samples_per_events
        self.window = signal.gaussian(8, std=3)
        self.convolution = convolution
        self.scaler = MinMaxScaler(feature_range=(0,1))

        assert seq_length % 2 == 0, "Please pass an even seq length"

    def __iter__(self):

        # Initialise Counter for events and files
        self.file_nr = 0
        self.event_in_file = 0
        self._sample_nr = 0

        return self

    def __next__(self):
        # Reads the current file and looks for event
        df, fname = self.read_file(self.files[self.file_nr])  # could be cached so you dont read it anew every iteration
        events = df[df['FO'] == 1]  # true where an event occurs, false everywhere else, use as mask

        if events.shape[0] > 0:
            if self._sample_nr < self.SAMPLES_PER_EVENT:
                # just give back the current event again, with different sampling, until we have generated
                # SAMPLES_PER_EVENT such samples

                event_frame = events.iloc[self.event_in_file].name
                indx_data, input_data, output_data = self.sample_seq_around_event_frame(df, event_frame)
                self._sample_nr += 1
            else:
                self.event_in_file += 1  # work on the next event in this file
                self._sample_nr = 0  # reset for the next event

                # check whether we are done with this file
                # otherwise we return the next event on the beginning of the next iteration
                if self.event_in_file >= len(events):
                    self.file_nr += 1
                    # If there still are files to run, it resets the variables
                    if self.file_nr < len(self.files):
                        logging.info("File is complete. Going to new file...")
                        self.event_in_file = 0
                        self._sample_nr = 0
                        return next(self)
                    else:
                        # processed the last file, we are done
                        logging.info("File is complete. All files done. Stopping...")
                        raise StopIteration
                elif self.event_in_file < len(events):
                    event_frame = events.iloc[self.event_in_file].name
                    indx_data, input_data, output_data = self.sample_seq_around_event_frame(df, event_frame)
                    self._sample_nr += 1
        else:
            logging.info("No events detected")
            self.file_nr += 1
            self.event_in_file = 0
            self._sample_nr = 0
            return next(self)

        return indx_data, input_data, output_data, fname

    def sample_seq_around_event_frame(self, df, event_idx):

        if event_idx >= 4:
            start_idx = event_idx - randint(4, self.seq_length / 7.5)
            if start_idx > 0:
                end_idx = start_idx + self.seq_length
                if end_idx <= len(df):
                    indx = df.iloc[start_idx:end_idx]['ID']
                    input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                    output = df.iloc[start_idx:end_idx]['FO']
                elif end_idx > len(df):
                    end_idx = len(df)
                    start_idx = end_idx - self.seq_length
                    indx = df.iloc[start_idx:end_idx]['ID']
                    input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                    output = df.iloc[start_idx:end_idx]['FO']
            elif start_idx <= 4:
                start_idx = event_idx
                end_idx = start_idx + self.seq_length
                indx = df.iloc[start_idx:end_idx]['ID']
                input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                output = df.iloc[start_idx:end_idx]['FO']
                if end_idx <= len(df):
                    indx = df.iloc[start_idx:end_idx]['ID']
                    input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                    output = df.iloc[start_idx:end_idx]['FO']
                elif end_idx > len(df):
                    end_idx = len(df)
                    start_idx = end_idx - self.seq_length
                    indx = df.iloc[start_idx:end_idx]['ID']
                    input = df.iloc[start_idx:end_idx, 1:self.input_size + 1]
                    output = df.iloc[start_idx:end_idx]['FO']

        if self.convolution:
            output = signal.convolve(output, self.window, mode='same')
        input = self.scaler.fit_transform(input)
        indx = indx.to_numpy()
        
        assert input.shape[0] == output.shape[0] == self.seq_length
        return torch.tensor(indx),torch.tensor(input), torch.tensor(output)

    def read_file(self, f):
        df = pd.read_csv(open(f, "r"))
        fname = os.path.basename(f)
        if fname[0:2] == 'RT':
            df = df.drop([#'ID',
                          'class',
                          'RTOE_X', 'RTOE_Y', 'RTOE_Z', 'V_RTOE_X', 'V_RTOE_Y', 'V_RTOE_Z',
                          #'RHLX_X', 'RHLX_Y', 'RHLX_Z', 'V_RHLX_X', 'V_RHLX_Y', 'V_RHLX_Z',
                          #'RHEE_X', 'RHEE_Y', 'RHEE_Z', 'V_RHEE_X', 'V_RHEE_Y', 'V_RHEE_Z',
                          #'RPMT5_X', 'RPMT5_Y', 'RPMT5_Z', 'V_RPMT5_X', 'V_RPMT5_Y', 'V_RPMT5_Z'
                          ], axis=1)
        elif fname[0:2] == 'LT':
            df = df.drop([#'ID',
                          'class',
                          'LTOE_X', 'LTOE_Y', 'LTOE_Z', 'V_LTOE_X', 'V_LTOE_Y', 'V_LTOE_Z',
                          #'LHLX_X', 'LHLX_Y', 'LHLX_Z', 'V_LHLX_X', 'V_LHLX_Y', 'V_LHLX_Z',
                          #'LHEE_X', 'LHEE_Y', 'LHEE_Z', 'V_LHEE_X', 'V_LHEE_Y', 'V_LHEE_Z',
                          #'LPMT5_X', 'LPMT5_Y', 'LPMT5_Z', 'V_LPMT5_X', 'V_LPMT5_Y', 'V_LPMT5_Z',
                          ], axis=1)
        return df, fname

# Model

In [33]:
class Network(nn.Module):
    # TO DO
    def __init__(self, config):
        super(Network, self).__init__()

        # Model construct Configuration
        self.batch_size  = config.batch_size
        self.num_layers  = config.num_layers
        self.hidden_size = config.hidden_size
        self.device      = config.device

        self.lstm = nn.LSTM(config.input_size, self.hidden_size, self.num_layers, dropout=config.drop_out, batch_first=True,
                            bidirectional=True)
        self.linear = nn.Linear(self.hidden_size * 2, config.output_size, bias=True)
        torch.nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, x):
        hidden, cell = self.init_hidden()
        out, _ = self.lstm(x, (hidden, cell))
        logits = self.linear(out)

        return logits[:, :, -1]

    def init_hidden(self):
        weight = next((self.parameters())).data
        hidden, cell = (weight.new(self.num_layers * 2, self.batch_size, self.hidden_size).zero_().to(self.device),
                        weight.new(self.num_layers * 2, self.batch_size, self.hidden_size).zero_().to(self.device))
        return hidden, cell

# Training and Validation

In [34]:
class Trainer:

    def __init__(self, model, config,name):

        # System configuration
        self.device = config.device

        # Model Construction
        self.model = Network(config).float()
        self.model.load_state_dict(model)
        self.model.to(self.device)
        print(self.model)

        # Peak detection and Evaluation
        self.output_dir = config.output_dir
        self.png_dir = config.png_dir
        self.globaliter = 0
       
        # DataLoader
        self.test_loader = DataLoader(
            CoolDataset(r'/content/drive/MyDrive/iteration4/Test'f"/{name}", config.seq_length,config.input_size,config.samples_per_event, convolution=True), batch_size=config.batch_size, drop_last=True,
            shuffle=False)
        
    def test(self):
        saliency_np_temp = np.empty((0,12))
        markers=[#'TOE_X', 'TOE_Y', 'TOE_Z', 'V_TOE_X', 'V_TOE_Y', 'V_TOE_Z',
                 'HLX_X', 'HLX_Y', 'HLX_Z', 'V_HLX_X', 'V_HLX_Y', 'V_HLX_Z',
                 'HEE_X', 'HEE_Y', 'HEE_Z', 'V_HEE_X', 'V_HEE_Y', 'V_HEE_Z',
                 'PMT5_X','PMT5_Y','PMT5_Z','V_PMT5_X','V_PMT5_Y','V_PMT5_Z']

        for indx, data, target, filename in self.test_loader:
            saliency_new = np.empty((0,18))
            saliency_average_temp = np.empty((0,18))
            saliency_average_fianl = np.empty((0,18))
            data.requires_grad_()
            self.model.eval()

            data = data.to(self.device)
            predictions = self.model(data.float())
            predictions_max, predictions_max_index = torch.max(predictions, dim=1)
            predictions_max.mean().backward()

            saliency = data.grad.data.abs()
            pred = torch.sigmoid(predictions)
            for i in range(0, saliency.shape[0]-1):
                    
                if predictions_max_index[i] > 11 or predictions_max_index[i] < 139:
                      
                      saliency_new_draw = saliency[i]
                      fig = plt.figure(figsize=(20, 10), constrained_layout=True)
                      gs1 = fig.add_gridspec(nrows=2, ncols=2, left=0.05, right=0.48, wspace=0.05)
                      f1_ax1 = fig.add_subplot(gs1[:, :-1])
                      f1_ax2 = fig.add_subplot(gs1[:-1, -1])
                      f1_ax3 = fig.add_subplot(gs1[-1, -1])
                      im = f1_ax1.imshow(saliency_new_draw, aspect='auto', interpolation='nearest', cmap=plt.cm.Blues)
                      im_2 = f1_ax2.imshow(saliency_new_draw[(predictions_max_index[i]-10):(predictions_max_index[i]+10)], aspect='auto', interpolation='nearest',cmap=plt.cm.Blues)
                      f1_ax3.plot(pred[i].detach().numpy())
                      f1_ax3.plot(target[i].detach().numpy())

                      # set ticks for axis
                      # f1_ax1.set_yticks(np.arange(len(saliency_new_temp)))
                      f1_ax1.set_xticks(np.arange(saliency_new_draw.shape[1]))
                      f1_ax1.set_xticklabels(markers)
                      f1_ax1.set_title(f"{filename[i]} -- Saliency heatmap for the prediction")

                      f1_ax2.set_yticks(np.arange(len(saliency_new_draw[(predictions_max_index[i] - 10):(predictions_max_index[i] + 10)])))
                      f1_ax2.set_yticklabels(range((predictions_max_index[i] - 10).numpy(), (predictions_max_index[i] + 10).numpy()))
                      f1_ax1.set_xticks(np.arange(saliency_new_draw.shape[1]))
                      filename_temp = filename[i]
                      plt.savefig(os.path.join(self.png_dir, f"{filename_temp[0:-4]} -- SaliencyHeatmap.jpg"), format='jpg')
                      plt.close(fig)
                      if len(saliency[i][(predictions_max_index[i]-10):(predictions_max_index[i]+10)]) > 0:
                        saliency_average =  saliency[i][(predictions_max_index[i]-10):(predictions_max_index[i]+10)]

                        if i == 0:
                          saliency_average_new   = np.row_stack((saliency_new, saliency_average))
                          saliency_average_final = saliency_average_new
                        else:
                          saliency_average_temp = np.row_stack((saliency_average_temp, saliency_average))

                      saliency_final = np.row_stack((saliency_average_fianl, saliency_average_temp))

        saliency_df = pd.DataFrame(saliency_final, columns= markers)
        saliency_df.to_csv(os.path.join(self.output_dir, f"Saliency.csv"), index=False)

# Main

In [35]:
 class Config:

    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

def main(loaded_model, model_config, name):
    trainer = Trainer(loaded_model, model_config, name)
    trainer.test()

if __name__ == '__main__':

    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Running on the GPU")
    else:
        device = torch.device("cpu")
        print("Running on the CPU")

    # Calling Models
    names    = ["MidFoot", "ForeFoot", "Heel"]

    model_dir = r'/content/drive/.shortcut-targets-by-id/1D1ZlxwJeesVJ3NPzK7AzVApyX6KyqwBA/Deep_learning/Deep Learning/Results/LSTM/TO/3markers/HLXPMT5HEE/v1/models/'
    model_files = tuple(Path(f"{model_dir}/CheckPoint_DS/").glob("**/*pt"))

    for name in names:
      for i in range(len(model_files)):

          model_name = os.path.basename((model_files[i]))
          loaded_model = torch.load(f"{model_dir}/CheckPoint_DS/" + model_name,map_location=torch.device(device))

          output_dir    = f"{model_dir}/saliency/{name}/Results_final{i}/csv"
          png_dir       = f"{model_dir}/saliency/{name}/Results_final{i}/png"

          print(model_name)

          # Directory Settings
          if not os.path.exists(output_dir):
              os.makedirs(output_dir)
              os.makedirs(png_dir)

          model_config = Config(

              # General Configuration
              device=device,
              batch_size=32,
              seq_length=150,
              samples_per_event = 1,
              output_size = 1,

              # LSTM
              input_size=18,
              hidden_size=256,
              num_layers=2,
              lr=0.001,
              drop_out=0.3,

              output_dir=output_dir,
              png_dir=png_dir,
          )

          main(loaded_model, model_config, name)

Running on the CPU
TO-256-HS-2-NL-0.001-LR-0.3-DO-1-FScheckpoint.pt
Network(
  (lstm): LSTM(18, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Linear(in_features=512, out_features=1, bias=True)
)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_l

TO-256-HS-2-NL-0.001-LR-0.3-DO-2-FScheckpoint.pt
Network(
  (lstm): LSTM(18, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Linear(in_features=512, out_features=1, bias=True)
)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_l

TO-256-HS-2-NL-0.001-LR-0.3-DO-3-FScheckpoint.pt
Network(
  (lstm): LSTM(18, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Linear(in_features=512, out_features=1, bias=True)
)


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_l

TO-256-HS-2-NL-0.001-LR-0.3-DO-9-FScheckpoint.pt
Network(
  (lstm): LSTM(18, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Linear(in_features=512, out_features=1, bias=True)
)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_l

TO-256-HS-2-NL-0.001-LR-0.3-DO-10-FScheckpoint.pt
Network(
  (lstm): LSTM(18, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (linear): Linear(in_features=512, out_features=1, bias=True)
)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Attempting to set identical bottom == top == -0.5 results in singular transformations; automatically expanding.
/usr/local/lib/python3.7/dist-packages/ipykernel_l